# Tutorial 0d 
## Cloning a case*

The `run_neon` script we used in the `Day0b` created and ran a base case as well as a `.transient` case. 

You'll recall that building the case was pretty time consuming. Luckily, if you aren't changing the model code, you can run from this base case without rebuilding multiple times. 

<br>

---

## In this tutorial

The tutorial has several components. Below you will find steps to: 
1. Create a clone for a new NEON site from the base case you already built.
2. Introduce ways to customize the configuration of your cases.

*Extra credit* 

3. Create another clone for an experiment you're hoping to do.

<div class="alert alert-block alert-warning">

<b>NOTICE:</b>  This tutorial assumes that you've done your homework! 
    
If you haven't downloaded CTSM from the github repository you need to go back to the: 
<ul>
    <li><b>Day0a_GitStarted tutorial</b> and </li> 
    <li><b>Day0b_NEON_Simulation_Tutorial</b> </li> 
</ul>
    
Do these first!

    
</div>


<div class="alert alert-block alert-warning">

<b>NOTICE:</b> If you're running this notebook through the NCAR JupyterHub login you need to be on a Cheyenne login node (NOT Casper).  

</div>


<div class="alert alert-block alert-info" markdown="1">

<b>TIP:</b>  Before we get started, make sure you're in a bash kernel 
<ul>
    <li> Switch kernal (upper right of your current notebook)</li>
    <li> Select either one of the Bash Kernels from the pop-up window</li>    
    <li> Click select</li>    
</ul>
</div>

<div>
<img src="https://github.com/NCAR/CTSM-Tutorial-2022/raw/main/images/kernel.png" width="670" />
</div>


<div class="alert alert-block alert-info" markdown="1">

<b>HINT:</b>  Most of the examples in this tutorial can be run directly from code 
cells of this notebook.  

It's also helpful to have a terminal window open to run from the command line.
To open a tab with a terminal window connection 


First you need to open a terminal window within CESM-Lab:
<ol>
    <li> Click on the + symbol in the upper left for a <i>New Launcher</i></li>
    <li> Click on the <i>Terminal</i> icon</li>
</ol>
</div>


<div>
    <img src="https://github.com/NCAR/CTSM-Tutorial-2022/raw/main/images/LaunchTerminal.png" 
         style="width:655px; height:375px;" />
</div>

***

<h1> 1. Create a clone </h1>

CTSM cases can be cloned to save time and create model experiments for different sites or different configurations that are otherwise identical to each other.   

## 1.1 Navigate to your source code

In [ ]:
cd ~/CTSM/tools/site_and_regional

## 1.2 Create a clone with `run_neon.py`
We'll just use run_neon again, but point to the **base case** you created in that `Day0b` tutorial.

**NOTE: if you're running on Cheyenne**, you may need to uncomment the the following two lines of code. 

This will help set up your conda environment correctly.  *This is not required if your running CESM-Lab in the cloud.*

In [ ]:
#module purge
#module load conda ncarenv

The following code will:
- create a clone, 
- download input data from NEON
- submit the simulation

As before, it takes a little bit for all this to happen, but you'll notice it's MUCH faster since we don't have to build the model.

In [ ]:
# Change the 4-character NEON site below.
export base_case='KONZ'   # should match the base case you created in Day0b
export new_case='CPER'    # the new site you want to run

In [ ]:
# then run_neon
./run_neon.py --neon-sites $new_case  \
   --output-root ~/scratch/NEON_cases \
   --base-case ~/scratch/NEON_cases/$base_case \
   --overwrite

Two new flags are being used here:
- `--base-case` points to base case that's already been built, letting you run your new simulation more quickly.  
- `--overwrite` will let you overwrite an existing case.  

<div class="alert alert-block alert-warning">

<b>NOTICE:</b> 
    
- Creating clones while pointing to a `--base-case` saves time by sharing the built model executable.
- Creating clones like this also requires that your base-case and new-case are the same type of simulation, `.transient` simulations in this example.  
- It also assumes that you're <b>not making modifications to the model source code </b> that would require you to rebuild a case.  

</div>


You can check to make sure things are running as expected. On the command line of a terminal window you can enter (HINT, you'll likely have to define `new_case` in the terminal window, as above in the notebook: 
> `ls ~/scratch/NEON_cases/$new_case.transient/run` **or** 

>`qstat -u $USER` **or**

>`tail  ~/scratch/NEON_cases/${new_case}.transient/CaseStatus`

<div class="alert alert-block alert-info" markdown='1'>
How are our simulations different?  let's take a quick look to see.
</div>

----
# 2. Introduction to controling case configuration

There are a few places that summarize differences in simulations.  These files include:
- `env_run.xlm`, which are variables that can be changed to configure your case;  
- `lnd_in`, which cannot be modified directly, but that gets information from user_nl_clm (and provides addition control over the way the land model is configured in your simulation);  
- `datm.streams.xml`, which also cannot be modified directly, but modified with user_datm_streams to point to different data atmosphere streams.

The easiest way to do this is by differencing the same files in each case directory with the `diff` command. 

## 2.1 `env_run.xml` 

In [ ]:
cd ~/scratch/NEON_cases
diff $new_case.transient/env_run.xml $base_case.transient/env_run.xml 

**What are the differences between these two cases**, based on their `env_run.xlm` files?

These differences are expected, because NEON cases are set up with `usermod_dirs` that control some of these .xml variables.

You can look at particlar changes for your $new_case below.

In [ ]:
cat ~/CTSM/cime_config/usermods_dirs/NEON/$new_case/shell_commands

We won't really go into usermod directories here, but they are a useful way to modify the way cases are configured for different model experiments.

usermod_dirs were used when you created your NEON cases with run_neon.  
- The shell_command and user_nl_* files are copied from `~/CTSM/cime_config/usermods_dirs/NEON` into your case directory.  
- Shell commands are executed after the case is created.
- This makes is easier set up cases in a consistent, repeatable way.
**Pretty slick software engineering!**

Some NEON sites have additional changes, notably for sites with gaps in the NEON input data.  These sites have shorter run times (than the 2018-2021 defaults).  

In [ ]:
cat ~/CTSM/cime_config/usermods_dirs/NEON/MOAB/shell_commands

These .xml changes set up high-level control over how your simulation is run.  What are some of the specifics related to the land model?  We can see this by looking at our `lnd_in` file.

## 2.2 `lnd_in` 

**What are the differences between these two cases**, based on their `lnd_in` files?

In [ ]:
diff $new_case.transient/CaseDocs/lnd_in $base_case.transient/CaseDocs/lnd_in 

<style> 
table td, table th, table tr {text-align:left !important;}
</style>
<div class="alert alert-block alert-info">
<b>REMEMBER:</b>
    
- The <i>lnd_in</i> file provides a high level summary of all the name list chagnes and files that are being used by CLM. 
- It can be found in the CaseDocs directory, or in your run directory. 
- You cannot directly modifiy the <i>lnd_in file</i>, instead users can modify <i>user_nl_clm</i>.

</div>

**Initial conditions dataset:** `finidat`
  - These are initial conditions files that we created by spinning up the model.  
  - Spin up requires starting the model from bare ground conditions (we call it a *coldstart*).
  - Spin up takes a few hundred years of simulations so that ecosystem carbon and nitrogen pools acheive steady state conditions (e.g. average net ecosystem exchange equals zero).  
  - Since this takes a long time, we provide initial conditions for you to start from.
  - **This also means that if you change model parameterizations, input data, or anything else you ahve to spin up the model again!** 

**Surface dataset:** `fsurdat`
  - The surface datasets describe what vegetation is growing in a grid cell, characteristics of soil physical properties, and much more information about what the land surface 'looks like' to the model.  
  - We modifed these the default surface dataset for each NEON simulation with information about the dominant plant functional type (PFT) and soil properties, based on NEON measurements. 
  - This could likely be further refined, but it's a step towards making the model look more like the real world ecosystems we are trying to simulate.


There are other differences in the build, but this is basically just reflecting the different case directories for the two NEON cases.

---

The `lnd_in` files are controlled by `user_nl_clm`.  Let's see how these are different.

In [ ]:
diff $new_case.transient/user_nl_clm $base_case.transient/user_nl_clm

It looks like the only difference here are initial conditions, but that's because we used environmental variables to get the right surface dataset

If you open one of the `user_nl_clm` files you'll see:
```
fsurdat = "$DIN_LOC_ROOT/lnd/clm2/surfdata_map/NEON/surfdata_1x1_NEON_${NEONSITE}_hist_78pfts_CMIP6_simyr2000_c230111.nc"
```

We saw this already by `diff`ing the `env_run.xml files`, above, but now we'll use `.xmlquery` to see how these are different in each case.

In [ ]:
echo moving to base_case directory
cd ~/scratch/NEON_cases/$base_case.transient/
./xmlquery NEONSITE

echo moving to new_case directory
cd ~/scratch/NEON_cases/$new_case.transient/
./xmlquery NEONSITE


You can also see what **parameter file** is being used for your case.  Since we haven't changed this, the model just points to the default CTSM5.1 parameter file.

In [ ]:
cat ~/scratch/NEON_cases/$base_case.transient/CaseDocs/lnd_in | grep paramfile
cat ~/scratch/NEON_cases/$new_case.transient/CaseDocs/lnd_in | grep paramfile


The land model is using site specific initial conditions and surface data for each NEON site.  How else are our simulations different?  

## 2.3 `datm.streams.xml` 

**What are the differences between these two cases**, based on their `datm.streams.xml` files?

The answer here isn't very interesting... the two cases likely point to different input data reflecting local meterology at each site.  It's still helpful to know about how these files are set up.


In [ ]:
cd ~/scratch/NEON_cases/
cat $new_case.transient/CaseDocs/datm.streams.xml | head -20

*Which aspects of this file could be changed for a different site?*

You can see check with this code *(HINT: you'll have to paste it in the command line on into a code cell).*

> ```
> diff $new_case.transient/CaseDocs/datm.streams.xml         $base_case.transient/CaseDocs/datm.streams.xml 
> ```

<div class="alert alert-block alert-info" markdown='1'>
<b>REMEMBER:</b> 

- The <i>datm.stream.xml</i> file points to all of the atmospheric boundary conditions (input data) that are being read in for a case. 
- Like your <i>lnd_in</i> files, it can be found in the CaseDocs directory, or in your run directory. 
- You cannot directly modifiy this file, instead users can modify <i>user_nl_datm_streams</i>.  

</div>

---

<div class="alert alert-success">
<strong>Congratulations!</strong> 
    
You have now cloned a CTSM case to run a simulation at a new NEON tower site, check that yoe can locate the history files from this site and try to plot up some of these data for these new results.
</div>

---

# 3. Create an experimental clone:

<h4> This step is optional, but provides helpful information that you may use in your own workflow </h4>

So far, everything we've done has been *out of the box* looking at different NEON sites, but without changing anything in the underlying model code.  You may want to do model experiments where you alter the vegetation growing at a site, modify some of the model parameters, modify namelist settings, change the input data, or even alter model code.  We'll get into how do make these changes later, but for now we'll get a test case set up.

Since we're already run an out of the box case for KONZ, we can create a paired experimental case at the same site.

<div class="alert alert-block alert-info" markdown='1'>
<b>RECOMMENDATION:</b> use a short, descriptive name for your experiment, it will help you down the road.
</div>


This example below just builds on what you've already been doing:

In [ ]:
# Change the 4-character NEON site below.
cd ~/CTSM/tools/site_and_regional
export base_case='KONZ'   # should match the base case you created in Day0b
export new_case='KONZ'    # the new site you want to run

# then run_neon
./run_neon.py --neon-sites $new_case  \
   --output-root ~/scratch/NEON_cases \
   --base-case ~/scratch/NEON_cases/$base_case \
   --overwrite \
   --experiment TestMod1 \
   --setup-only  

Two new flags are being used here:
- `--experiment` just appends the case name for the experiment
- `--setup-only` will create the case, but not submit it.

<div class="alert alert-block alert-warning" markdown='1'>

<b>WARNING:</b> Because we're also using the `--base-case` flag, we won't have to rebuild our experimental case.  This may not be advisable if you're modifying model code.

</div>

At this point your experimental case has been created.  
- What is the case name?
- Can you navigate to your case directory?
- Are there any differences between this experimental case and the base case you already ran?
- Can you find the datm input data for your case?
- Is the model going to use an initial conditions file?
- Where is the surface dataset that's being used?
- Can you find the parameter file for your case?

**Extra Credit**
- What changes may you like to make to the model in this new experimental case?

In [ ]:
# Change the 4-character NEON site below.
cd ~/CTSM/tools/site_and_regional
export base_case='KONZ'   # should match the base case you created in Day0b
export new_case='CPER'    # the new site you want to run

# then run_neon
./run_neon.py --neon-sites $new_case  \
   --output-root ~/scratch/NEON_cases \
   --base-case ~/scratch/NEON_cases/$base_case \
   --overwrite \
   --experiment PRISM \
   --setup-only  

### XML changes required for PRISM 
We've set things up so with a single XLM change you can get PRISM precipitation for NEON sites in the conterminous United State (CONUS)


In [ ]:
cd ~/scratch/NEON_cases/ONAQ.PRISM.transient
./xmlquery CLM_USRDAT_NAME
./xmlchange CLM_USRDAT_NAME=NEON.PRISM 
./xmlquery CLM_USRDAT_NAME


**TODO** we still need to figure out how to handle restart files for these cases.


- **`./xmlquery`** asks what's currently being used for an xml variable

- **`./xmlchange`** sets this to a new value (here so that CLM_USRDAT_NAME is changed from NEON to NEON.PRISM), as confirmed by the second `./xmlquery`


### Namelist changes needed for PRISM
run_neon automatically gets a restart file for `.transient` cases from the NEON input data and set this in your user_nl_clm

You can confirm this with the following command.

In [ ]:
cat user_nl_clm | tail -1

You'll notice that even though your case directory is named as expected `ONAQ.PRISM.transient`, the actual restart files doesn't share the **ONAQ.PRISM** case name (`ONAQ.2023-01-13.clm2.r.0318-01-01-00000.nc`).




In [ ]:
./query_config --compsets clm

The long name for the compset used here is `HIST_DATM%1PT_CLM51%BGC_SICE_SOCN_SROF_SGLC_SWAV_SESP`, which defines
   - time = `HIST_` (vs. 1850, HIST, SSP, etc)
   - data atmosphere `DATM%1PT_`, from a single point, as opposed to model atmosphere (e.g., CAM) 
   - land model `CLM51%BGC_` CLM5.1 physics package with active biogeochemistry (BGC)
   - stub sea ice model `SICE_`
   - stub ocean model `SOCN_` (see there's no active ocean model!)
   - stub river model `SROF_` (other options include MOSART, MIZUROUTE or RTM)
   - stub glacier `SGLC_`
   - stub wave model `SWAV_`   

<div>
<img src="https://github.com/NCAR/CTSM-Tutorial-2022/raw/main/images/Components.png" width="460" height="460" />
</div>


Key Definitions:
- **Active:** Simulation is using the code from the model during the run
- **Data:** Simulation is reading in data from a file for this component
- **Stub:** Component is not being used
<br><br> 

<div class="alert alert-block alert-warning" markdown="1">

<b>HINT:</b> Some compsets are "scientifically supported" and others are not. A scientifically supported compset just means that we've done some additional testing and evaluation with that compset.  You can use an unsupported compset (which is encouraged!), but will need to add the option <i>--run_unsupported</i> at the end of the <i>create_newcase</i> command line.

</div>


<div class="alert alert-block alert-info" markdown="1">

<b>TIP:</b> More information on model <a href="https://www.cesm.ucar.edu/models/cesm2" target="_blank">Configurations and Grids can be found on  the CESM website</a> (see <i>Configurations and Grids</i> subheading at the bottom of the page)

</div>



***
<h2> 1.2 Invoke case.setup </h2>

*This step configures the model so that it can compile* 
***

### 1.2.1 Move to your case directory

In [ ]:
cd ~/NEON_cases/${neon_site}.transient

As before, this doesn't always render properly online:

`~/NEON_cases/${neon_site}.transient`

### 1.2.2 Set up your case

The `./case.setup` script:
1. configures the model
2. creates files to modify input data and run options

In [ ]:
./case.setup

Using this command, we just configured the model and created the files to modify options & input data.
<div>
<img src="https://github.com/NCAR/CTSM-Tutorial-2022/raw/main/images/CaseSetup.png" width="670" />
</div>

***
<h2> 1.3 Build the executable  </h2>

*This step compiles the model*

It also takes a long time, so be patient.
***

The `./case.build` script:
1. Checks input data
2. Creates a build/run directory with model executable and namelists

TODO: using qcmd -- ./case.build fails with wget errors of met data 

In [ ]:
./case.build


---
This takes some time, and will throw a bunch of errors... don't worry, just give it time


**When the build completes successfully you'll see a notice that the `MODEL BUILD HAS FINISHED SUCCESSFULLY`**


<div class="alert alert-block alert-warning">
   
<b>HINT:</b>  Building a NEON case requires that you have the `listing.csv` file in your `~/NEON_cases` directory.  
<p> Don't worry, you have this if you did the <b>Day0b tutorial</b>  <span>&#x1F609</span> </p>

</div>

You can read on, but before executing any code blocks in the notebook **wait for the model to build.**
This can take a while, especially while you're wating for your `qcmd` job to start and as code for the land model compiles.

<style> 
table td, table th, table tr {text-align:left !important;}
</style>
<div class="alert alert-block alert-info">

<b>NOTE:</b> The command <i>qcmd -- ./case.build</i> is specific for NCAR environments, including Cheyenne and cloud configurations, and runs the command on a computing node, reducing the load on the login node. <b>You must include <i>qcmd --</i> when running on Cheyenne</b>, and it's highly advised on shared cloud systems too.  On single-user cloud systems, it isn't needed, though it may speed up builds.

</div>

---

### 1.3.2 Customize your case by modifying user_nl_clm

Namelist are one way you can customize setting of your case. 

Namelist changes can be made after the model is built, which can save you time down the road!

We'll explore these features more in upcoming tutorials, but for now we just need to add an initial conditions file ( or restart file) to your case.

This is done automatically by the `run_neon` script that we used in the *Day0b tutorial*. If you aren't making code or parameter changes, you can use the restart files that we've prestaged on the NEON server.  These can be found in the *listing.csv* file. 

The command below will:
- use `awk` and `grep` to get the url of the restart file on NEON's servers.  
- `tail -1` gives us the last (most recent) file in the list
*Note, this is NOT common knowledge, at least for me.  I spent > an hour figuring this out!* 

In [ ]:
awk -F "," '{print $1}' ~/NEON_cases/listing.csv | grep lnd/ctsm/initdata/${neon_site} | tail -1

Now we'll use `wget` to copy this file from the NEON servers into our run directory

*NOTE* run_neon handles this a bit differently. 
- **OPTIONAL EXTRA CREDIT** Can you see where the restart file is in your `Day0b` example?

In [ ]:
wget -P ~/scratch/NEON_runs/${neon_site}.transient/run/ \
    eval $(awk -F "," '{print $1}' ~/NEON_cases/listing.csv | grep lnd/ctsm/initdata/${neon_site} | tail -1)
   

Finally we'll add the path to this restart to our user_nl_clm file

First print the file name

In [ ]:
ls ~/scratch/NEON_runs/${neon_site}.transient/run/${neon_site}*.clm2.r.*

<div class="alert alert-block alert-warning">
   
<b>WARNING:</b>  You have to modify the code block below by pasting the string that prints from the cell above into the part that says XXX below
</div>

Here's an example of what I mean:

```
echo "finidat='/glade/u/home/wwieder/scratch/NEON_runs/ONAQ.transient/run/ONAQ.2023-01-13.clm2.r.0318-01-01-00000.nc' " >> user_nl_clm
```

In [ ]:
echo "finidat = 'XXX' " >> user_nl_clm

Open your user_nl_clm file.  

**Are you pointing to the initial conditions file copied from NEON?**

If so, let's move on.

<h3> 1.4 Submit your case  </h3>

Now you're ready to submit the case!

*This step submits the model simulation*
- You'll also be downloading all the meterological data from NEON for your site, this also takes a little time and prints lots of information to the screen
***

In [ ]:
./case.submit

When you submit a job, you will see confirmation that it successfully submitted:

### Congratulations! You've created and submitted a single point NEON run from scratch.

Next, you will probably want to check on the status of your jobs.

<div class="alert alert-block alert-info" markdown="1">

<b>TIP:</b> This is dependent on the scheduler that you're using. 

<ul>
<li>Cheyenne uses <b>PBS</b> where status is checked with <i>qstat -u $USER</i></li>
<li>This is also enabled in the cloud for you, try it in the code block below</li></ul>

If you want to stop the simulation, you can do so with qdel here (or on Cheyenne).
<ul>
<li> Find your Job ID after typing <i>qstat</i> </li>
<li> Type <i>qdel {Job ID}</i> </li>
</ul>
</div>

In [ ]:
qstat -u $USER

---
Once your jobs are complete (or show the 'C' state under the 'Use' column, which means complete), we can check the CaseStatus file to ensure there were no errors and it completed successfully.  To do this, we'll 'tail' the end of the CaseStatus file:

In [ ]:
tail  ~/NEON_cases/${neon_site}.transient/CaseStatus

You should see several lines, with the middle one saying 'case.run success'.  

Before that you'll see notifications about xml changes, case.setup, and case.submit, and case.run

<div class="alert alert-success">
<strong>Congratulations!</strong> 
    
You've created a CTSM case for the NEON tower you selected.

We'll build on these basics in additional tutorials to customize your simulations.
</div>




****
# 2. Locate model history files
Your simulation will likely take some time to complete. The information
provided next shows where the model output will be located while the
model is running and once the simulation is complete. We also provide
files from a simulation that is already complete so that you can do the
next exercises before your simulation completes. 

<div class="alert alert-block alert-info" markdown="1">

<b>When your simulation is running</b> history files go to your scratch directory: 

<ul>
<li><i>~/scratch/NEON_runs/{CASE}</i> </li>
</ul>

Within this directory you can find <i>/run</i> and <i>/bld</i> subdirectories.

<b>When the simulation is complete</b>, a short-term archive directory is created, and history files are moved here: 

<ul>
<li><i>~/scratch/NEON_runs/<b>archive</b>/{CASE}/lnd/hist/</i> </li>
</ul>

Note that files necessary to continue the run are left in the run directory: <i>~/scratch/NEON_runs/{CASE}/run</i>
</div>

## 2.1 Run directory
*What's in your run directory?*

In [ ]:
ls ~/scratch/NEON_runs/${neon_site}.transient/run

*Do you see any log or history files?*
- log files look like `lnd.log*`
- history files look like `${neon_site}.transient.clm2.h0.*.nc`

You can keep running the cell above until you see log and history files, then the model is running.

---

## 2.2 Archive directory
*What's in your archive directory?*

In [ ]:
ls ~/scratch/NEON_runs/archive/${neon_site}.transient/lnd/hist | head -5

You can drop the `| head -5` part of the cell above if you want to see ALL the files that have been archived.

If you don't see any history files your simulation is likely still running or in the queue (check using squeue or qstat). Check again before you leave today to see if your simulation completed and if the files were transferred to archive. Even if your run isn't finished, you can move on in this tutorial.

If you'd like to visualize these results you can go back to tutorial Day0c and modify to point to these results from the neon_site you just ran.

***
# 3. Create a Clone
<h4> This step is optional, but provides helpful information </h4>
    
Creating and building a new case is slow.  

Cloning cases can speed this up!

<h2>3.1 Create a clone </h2>

- Clones use the same resolution, compset, and output root.
- Clones can also share the same executable (build), which can save time!

<div class="alert alert-block alert-warning" markdown="1">

<b>WARNING:</b> If you're making code modifications be careful using clones that share the same build. The example below is a good way to run the same model configuration at different NEON sites.

</div>



In [ ]:
# Move back to your source code directory
cd ~/CTSM/cime/scripts

# Change the 4-character new NEON site you want to run.
export neon_site2="TEAK"

# then create a cloned case.
./create_clone --case ~/NEON_cases/${neon_site2}.transient --clone ~/NEON_cases/${neon_site}.transient --user-mods-dirs ~/CTSM/cime_config/usermods_dirs/NEON/${neon_site2} --keepexe

As before, the code above doesn't always render properly online.  Here's what we're doing:

`./create_clone --case ~/NEON_cases/${neon_site2}.transient --clone ~/NEON_cases/${neon_site}.transient --user-mods-dirs ~/CTSM/cime_config/usermods_dirs/NEON/${neon_site2} --keepexe`
***
### **./create_clone**

<div class="alert alert-block alert-info">

<b>NOTE:</b> There is a lot of information that goes into creating a clone.

You can learn more about the options by typing <i>./create_clone --help</i> on the the command line or in a new code cell.

<b>We'll briefly go over some of the highlights here.</b>

</div>

---

### Required arguments to create a clone
There are 2 required arguments needed to create a clone.  These include: 
1. `--case`, this is the path and name of the new case your cloning.
<br><br>
2. `--clone`, this is the path and name of the existing case you want to clone.
<br><br>  
### Additional information we provided here were: 
3. `--user-mods-dirs` as before, this sets up the configuration of the case with a user modification directory that defines the location of the new site we're running, but it requires the full path to the `user-mods-dir` you want to use.
<br><br>  
4. `--keepexe` Point to original build from the case being cloned.

***
<h2> 3.2 Modify namelists </h2>

*Because you created a clone and point to an existing build we can skip the `case.setup` and `case.build` steps* 
***

### 3.2.1 Move to your case directory 

In [ ]:
cd ~/NEON_cases/${neon_site2}.transient

### 3.2.2 Make namelist changes
We'll use the same series of commands as before to get the restart file into our run directory

In [ ]:
wget -P ~/scratch/NEON_runs/${neon_site2}.transient/run/ \
    eval $(awk -F "," '{print $1}' ~/NEON_cases/listing.csv | grep lnd/ctsm/initdata/${neon_site2} | tail -1)
   

Print the file name for the restart file

In [ ]:
ls ~/scratch/NEON_runs/${neon_site2}.transient/run/${neon_site2}*.clm2.r.*

And now paste it into `user_nl_clm`  

**REMEMBER** you need to replace XXX with the results of the cell above

In [ ]:
echo "finidat = 'XXX' " >> ~/NEON_cases/${neon_site2}.transient/user_nl_clm

***
<h2> 3.3 Submit the case </h2>

*Because you created a clone and point to an existing build we can skip the `case.setup` and `case.build` steps* 
***

- As before, you'll be downloading all the meterological data, which takes a little time and prints lots of information to the screen
***


In [ ]:
./case.submit

<div class="alert alert-success">
<strong>Congratulations!</strong> 
    
You've created and run a clone of CLM for the NEON tower you selected.
</div>


You can track progress on your run using steps outlined in part #2 of this tutorial.
You can also work on visualizing your results for different sites using code provided in `Day_0c` materials.




If you're developing this tutorial:
Before saving and pushing this code to github go to `Kernel` and `Restart kernel and clear all outputs...`